# 建構一個 Chatbot:

此範例是根據[這個](https://python.langchain.com/v0.2/docs/tutorials/chatbot/)改編

我們將透過一個範例來說明如何設計和實現由LLM支援的Chatbot。該Chatbot將能夠進行對話並記住先前的互動。

請注意，我們建立的這個Chatbot將僅使用語言模型進行對話。您可能正在尋找其他幾個相關概念：

- [會話式 RAG](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/)：透過外部資料來源實現Chatbot體驗。

- [Agent](https://python.langchain.com/v0.2/docs/tutorials/agents/)：建構一個可以採取行動的Chatbot。

本教程將涵蓋對這兩個更高級主題有幫助的基礎知識，但如果您選擇，可以直接跳到那裡。

In [ ]:
!pip install langchain
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
import getpass
import os
from langchain_openai import ChatOpenAI

os.environ["LANGCHAIN_TRACING_V2"] = "true"
# 替換為你的LANGCHAIN_API_KEY
os.environ["LANGCHAIN_API_KEY"] = "替換為你的LANGCHAIN_API_KEY"

os.environ["OPENAI_API_KEY"] = "替換為你的OPENAI_API_KEY"


model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好 我是飛魚")])


AIMessage(content='你好飛魚，很高興能和你聊天。有什麼可以幫助你的嗎？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 17, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-971d220b-7ed4-4c25-93c6-7203501457d1-0', usage_metadata={'input_tokens': 17, 'output_tokens': 37, 'total_tokens': 54})

API 參考：[HumanMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html)
該模型本身沒有任何狀態概念。例如，如果您提出後續問題：
model.invoke([HumanMessage(content="叫我的名字？")])

In [ ]:
model.invoke([HumanMessage(content="我叫啥?")])

AIMessage(content='抱歉，我无法知道你的名字，因为我是一个虚拟助手。您可以告诉我您的名字，我们可以继续交流。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 13, 'total_tokens': 61}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7170be91-6477-4c0a-8d55-729b6ce88943-0', usage_metadata={'input_tokens': 13, 'output_tokens': 48, 'total_tokens': 61})

可以看出，它沒有將前面的對話轉入上下文，無法回答問題。這會帶來糟糕的聊天機器人體驗！

為了解決這個問題，我們需要將整個對話歷史記錄傳遞到模型中。讓我們看看這樣做時會發生什麼：



In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)



AIMessage(content='Your name is Bob. How can I assist you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 35, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8c5dca86-adaf-4bc9-8b62-1990c682f667-0', usage_metadata={'input_tokens': 35, 'output_tokens': 14, 'total_tokens': 49})

上方為官方的教學，現在我們可以看到我們從GPT那得到還可以的回應！

這就是聊天機器人對話互動能力的基本想法。

不過假如改成中文會有怎樣的效果，效果如下：

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好 我是飛魚"),
        AIMessage(content="你好飛魚，很高興能和你聊天。有什麼可以幫助你的嗎？"),
        HumanMessage(content="我的名字是?"),
    ]
)

AIMessage(content='抱歉，我不知道你的名字。你可以告訴我你的名字嗎？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 67, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f734f6d1-ad04-4ffa-8b2a-82365cd96580-0', usage_metadata={'input_tokens': 67, 'output_tokens': 28, 'total_tokens': 95})

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好 我的名字叫飛魚"),
        AIMessage(content="你好飛魚，很高興能和你聊天。有什麼可以幫助你的嗎？"),
        HumanMessage(content="我的名字是?"),
    ]
)

AIMessage(content='您剛才說您的名字叫飛魚。您想要我們用另一個名字來稱呼您嗎？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 71, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a417d8a3-9f3c-4133-9da7-601ebef41f7b-0', usage_metadata={'input_tokens': 71, 'output_tokens': 45, 'total_tokens': 116})

從上面可以看到，在繁體中文方面可能因為沒有那麼多的資料，又或者因為沒有那麼多的微調或RLHF，所以必須寫得特別直接才有正確的回應，不過這只是GPT 3.5的情況，或許這會在GPT4或之後的版本就改善了。

# 歷史訊息

我們可以使用訊息歷史記錄類別來包裝我們的模型並使其具有狀態。這可用於追蹤模型的輸入和輸出，並將它們儲存在某個資料儲存中。未來的互動將載入這些訊息並將它們作為輸入的一部分傳遞到運作中。讓我們看看如何使用它！

首先，讓我們確保安裝langchain-community，因為我們將使用其中的整合來儲存訊息歷史記錄。

In [ ]:
! pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

API 參考：[BaseChatMessageHistory](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html) |[記憶體中聊天訊息歷史記錄](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.InMemoryChatMessageHistory.html)| [RunnableWithMessageHistory](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)


我們現在需要建立一個 `config`  每次傳遞到可運行物件中的物件。此配置包含的資訊不是直接輸入的一部分，但仍然有用。在本例子中，我們需要包含一個 `session_id` .這應該看起來像：

In [ ]:
config = {"configurable": {"session_id": "abc1"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content


'Your name is Bob. How can I help you today, Bob?'

In [56]:
config = {"configurable": {"session_id": "abc2"}}
response = with_message_history.invoke(
    [HumanMessage(content="你好 我的名字是取把勾")],
    config=config,
)
response = with_message_history.invoke(
    [HumanMessage(content="我的名字是? ")],
    config=config,
)

response.content

'非常抱歉，我无法得知您的真实姓名。如果您有其他问题或需要帮助，请随时告诉我。我会尽力提供支持。'

In [53]:
model = ChatOpenAI(model="gpt-4o")
config = {"configurable": {"session_id": "abc3"}}
response = with_message_history.invoke(
    [HumanMessage(content="你好 我的名字是取把勾")],
    config=config,
)
response = with_message_history.invoke(
    [HumanMessage(content="我的名字是? ")],
    config=config,
)

response.content


'抱歉，我无法直接获取你的名字。请问有什么问题或者需要帮助的吗？'

看來這方面就算改成 gpt-4o的模型也沒有改變。

In [54]:
model = ChatOpenAI(model="gpt-3.5-turbo")
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I don't have access to personal information like your name. How can I assist you today?"

然而，我們是可以回到原來的對話（因為其對話保存在資料庫中）

In [55]:
config = {"configurable": {"session_id": "abc1"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob. How can I assist you, Bob?'

In [57]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I don't have access to personal information such as your name. If you would like to share your name with me, feel free to do so. How can I assist you today?"

如此一來，我們便能讓聊天機器人同時與多位用戶暢談！(看來只能用英文)

目前，我們只是在模型周圍添加了一個簡單的持久層(存儲對話)。接下來，我們可以引入提示模板，使對話變得更加複雜和個性化。

# 提示模板

提示模板有助於將原始用戶資訊轉換為LLM可處理的格式。在當前情境中，原始用戶輸入僅僅是一條訊息，我們將其直接傳遞給LLM。現在，讓我們稍微複雜化一點。首先，讓我們添加一條包含自定義指令的系統訊息（但仍將訊息作為輸入）。接下來，我們將添加更多輸入，而不僅僅是訊息。

首先，讓我們添加一條系統訊息。為此，我們將創建一個 `ChatPromptTemplate`。我們將利用 `MessagesPlaceholder` 來傳遞所有訊息。


In [59]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello Bob! How can I assist you today?'

In [60]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

response = chain.invoke({"messages": [HumanMessage(content="你好! 我是飛魚")]})

response.content

'你好! 有什麼問題我可以幫忙解答嗎？'

API 參考：[ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) | [MessagesPlaceholder](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)

請注意，這稍微改變了輸入類型 - 現在我們不再傳入一個訊息列表，而是傳入一個字典，其中包含一個 `messages` 鍵，該鍵的值是一個訊息列表。


現在，我們可以像之前一樣，將其封裝在 Messages History 對象中。

In [62]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello Jim! How can I assist you today?'

In [63]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim.'

不過繁中方面感覺還是沒微調過

In [64]:
model = ChatOpenAI(model="gpt-3.5-turbo")
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc6"}}
response = with_message_history.invoke(
    [HumanMessage(content="你好！我是阿拉丁")],
    config=config,
)

response.content

'你好，阿拉丁！有什么可以帮助你的吗？'

In [65]:
response = with_message_history.invoke(
    [HumanMessage(content="你猜猜我是?")],
    config=config,
)

response.content

'抱歉，我无法猜测你是谁。您可以告诉我更多关于您的信息吗？我会尽力帮助您。'

現在讓我們來讓提示模板變得更為複雜一點。假設提示模板現在看起來像這樣：


In [66]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

請注意，我們在提示中新增了一個 `language` 輸入。現在，我們可以調用 chain並傳入我們選擇的語言。


In [67]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

response = chain.invoke(
    {"messages": [HumanMessage(content="你好！我是馬克")], "language": "Spanish"}
)

response.content

'¡Hola Mark! ¿En qué puedo ayudarte hoy?'

看來這個有微調過，或是有相關的資料集訓練過。

讓我們將這個更複雜的鏈封裝在 `Message History` 類中。這次，由於輸入中有多個鍵，我們需要指定正確的鍵來保存聊天歷史記錄。


In [68]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc11"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [69]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Tu nombre es Todd. ¿Hay algo más en lo que pueda ayudarte?'

In [72]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc12"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="你好！我是阿桃")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola! ¿En qué puedo ayudarte hoy?'

In [73]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="我叫啥?")], "language": "Spanish"},
    config=config,
)

response.content

'Tu nombre es "阿桃"。 ¿Hay algo más en lo que pueda ayudarte?'

這個中文是回答正確的，很神奇就是，可能是封裝的類裡面有設定相關的prompt，或者之前的系統prompt 有產生作用。

## 管理對話歷史

在構建聊天機器人時，一個重要的概念是要理解如何管理對話歷史。如果不加以管理，訊息列表會無限增長，可能超出 LLM 的上下文窗口。因此，添加一個步驟來限制傳入訊息的大小非常重要。

重要的是，您需要在**提示模板之前**執行此操作，但在**從訊息歷史記錄載入先前的訊息之後**執行。

我們可以在提示前添加一個簡單的步驟，以適當地修改 `messages` 鍵，然後將這個新的鏈封裝在 `Message History` 類中。

LangChain 提供了一些內建的輔助工具來**管理訊息列表( [managing a list of messages](https://python.langchain.com/v0.2/docs/how_to/#messages))**。在這種情況下，我們將使用  [trim_messages](https://python.langchain.com/v0.2/docs/how_to/trim_messages/)  輔助工具來減少我們發送到模型的訊息數量。此修剪器允許我們指定要保留多少個 token，以及其他參數，例如是否始終保留系統訊息以及是否允許部分訊息。


In [74]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

API [Reference:SystemMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html) | [trim_messages](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.utils.trim_messages.html)

要在我們的 chain 中使用它，我們只需要在將 `messages` 輸入傳遞給提示模板之前執行修剪器。

現在，如果我們嘗試詢問模型我們的名字，它將無法回答，因為我們修剪了聊天歷史記錄中的那一部分。


In [75]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to your name. How can I assist you today?"

API Reference: [RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html)

但如果我們詢問有關過去幾條消息中的資訊，它會記得：


In [76]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2?"'

現在，讓我們將其封裝在 Message History 中。


In [77]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't know your name."

如預期的那樣，我們一開始提到名字的第一條消息已被修剪掉了。此外，聊天歷史記錄中現在多了兩條新消息（我們最新的問題和最新的回應）。這意味著之前在我們的對話歷史中可訪問的更多資訊現在不再可用！在這種情況下，我們最初的數學問題也已從歷史記錄中修剪掉，因此模型不再知道它了：


In [78]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked me a math problem yet. Feel free to ask, and I'll do my best to help you with it."

## 串流傳輸

現在，我們已經有一個功能性的聊天機器人。然而，對於聊天機器人應用來說，一個非常重要的用戶體驗考量是串流傳輸。大型語言模型 (LLMs) 有時可能需要一段時間才能回應，因此為了改善用戶體驗，大多數應用程序會在每個 token 生成時就將其串流傳輸回來。這允許用戶看到進度。

實際上，這樣做非常簡單！

所有 chain 都公開了一個 `.stream` 方法，使用訊息歷史記錄的鏈也不例外。我們只需使用該方法來獲得串流回應即可。


In [79]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Sure|,| Todd|!| Here|'s| a| joke| for| you|:| Why| did| the| scare|crow| win| an| award|?| Because| he| was| outstanding| in| his| field|!| 😄||

後續步驟
既然您已經了解了在 LangChain 中創建聊天機器人的基礎知識，接下來您可能會對一些進階教程感興趣：

對話式 RAG： 使聊天機器人能夠與外部資料源進行交互。
代理： 構建一個可以採取行動的聊天機器人。
如果您想深入了解具體細節，以下是一些值得關注的內容：

串流傳輸： 串流傳輸對於聊天應用至關重要。
如何添加訊息歷史記錄： 深入了解與訊息歷史記錄相關的所有內容。
如何管理大型訊息歷史記錄： 更多管理大型聊天歷史記錄的技術。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')